# SSD: Single Shot MultiBox Detector

## 摘要  


该文章在既保证速度，又要保证精度的情况下，提出了 SSD 物体检测模型，与现在流行的检测模型一样，将检测过程整合到一个检测网络中。便于训练与优化，同时提高检测速度。SSD 将输出一系列离散化的 bounding boxes，这些 bounding boxes 是在不同层次（layers）上的 feature maps 上生成的，并且有着不同的 aspect ratio。在 prediction 阶段：要计算出每一个 default box 中的物体，其属于每个类别的可能性，即 score，得分。如对于 PASCAL VOC 数据集，总共有 20 类，那么得出每一个 bounding box 中物体属于这 20 个类别的每一种的可能性。同时，要对这些 bounding boxes 的 shape 进行微调，以使得其符合物体的 外接矩形。还有就是，为了处理相同物体的不同尺寸的情况，SSD 结合了不同分辨率的 feature maps 的 predictions。相对于那些需要 object proposals 的检测模型，本文的 SSD 方法完全取消了 proposals generation、pixel resampling 或者 feature resampling 这些阶段。这样使得 SSD 更容易去优化训练，也更容易地将检测模型融合进系统之中。在 PASCAL VOC、MS COCO、ILSVRC 数据集上的实验显示，SSD 在保证精度的同时，其速度要比用 region proposals 的方法要快很多。SSD 相比较于其他单结构模型（YOLO），SSD 取得更高的精度，即是是在输入图像较小的情况下。如输入 300×300 大小的 PASCAL VOC 2007 test 图像，在 Titan X 上，SSD 以 58 帧的速率，同时取得了 72.1% 的 mAP。
如果输入的图像是 500×500，SSD 则取得了 75.1% 的 mAP，比目前最 state-of-art 的 Faster R-CNN 要好很多。
  
SSD方法的贡献：  

提出了新的物体检测方法：SSD，比原先最快的 YOLO: You Only Look Once 方法，还要快，还要精确。保证速度的同时，检测效果 mAP 可以达到 region proposals 技术的方法（如 Faster R-CNN）等。  

SSD 方法的核心就是 predict object，以及其归属类别的 score；同时，在 feature map 上使用小的卷积核，去 predict 一系列 bounding boxes 的 box offsets。  

本文中为了得到高精度的检测结果，在不同层次的 feature maps 上去 predict object、box offsets，同时，还得到不同 aspect ratio predictions。  

本文的这些改进设计，能够在当输入分辨率较低的图像时，保证检测的精度。同时，这个整体 end-to-end 的设计，训练也变得简单。在检测速度、检测精度之间取得较好的 trade-off。  

本文提出的模型（model）在不同的数据集上，如 PASCAL VOC、MS COCO、ILSVRC，都进行了测试。在检测时间、检测精度上，均与目前物体检测领域 state-of-art 的检测方法进行了比较。

## 1 网络模型  

SSD 是一个前向传播 CNN 网络，产生一系列固定大小的 bounding boxes，以及每一个 box 中包含物体实例的可能性，之后进行非极大值抑制得到最终的 predictions。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/SSD.jpg?raw=true)
<center> **Figure 1** </center >
SSD 模型的最开始部分，本文称作 base network，是用于图像分类的标准架构。在 base network 之后，本文添加了额外辅助的网络结构：  

Multi-scale feature maps for detection   
在基础网络结构后，添加了额外的卷积层，这些卷积层的大小是逐层递减的，可以在多尺度下进行 predictions。  

Convolutional predictors for detection   
每一个添加的特征层（或者在基础网络结构中的特征层），可以使用一系列 convolutional filters，去产生一系列固定大小的 predictions。对于一个大小为 m×n，具有 p 通道的特征层，使用的 convolutional filters 就是 3×3×p 的 kernels。产生的 predictions，要么就是归属类别的一个得分，要么就是相对于 default box coordinate 的 shape offsets。  

default box 以及 feature map cell：  
feature map cell 就是将 feature map 切分成 5×5 大小的格子；  
而 default box 就是每一个格子上，一系列固定大小的 box  
如下图所示。在 5×5 大小的 feature map 上面，每个位置都生成3个不同大小的box，则该 feature map 对应的default box 共75个
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/SSD4.jpg?raw=true)
<center> **Figure 2** </center >
Default boxes and aspect ratios   
每一个 box 相对于与其对应的 feature map cell 的位置是固定的。在每一个 feature map cell 中，我们要 predict 得到的 box 与 default box 之间的 offsets，以及每一个 box 中包含物体的 score。   
因此，对于一个位置上的 k 个boxes 中的每一个 box，我们需要计算出 c 个类，每一个类的 score，还有这个 box 相对于它的默认 box 的 4 个偏移值（offsets）。于是，在 feature map 中的每一个 feature map cell 上，就需要有 (c+4)×k 个 filters。对于一张 m×n 大小的 feature map，即会产生 (c+4)×k×m×n 个输出结果。
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/SSD3.jpg?raw=true)
<center> **Figure 3** </center >
如图所示：对5×5大小的 feature map 有 75 个default box，则用（21+4）× 3 个filter 生成待预测的特征图之后，产生(21+4)×3×5×5 个输出结果

**如上图所示：SSD预测的目标就是以一张图中所有 default box (7308个)为窗口, 看其窗口是否存在物体, 如果有物体, 预测其类别以及位置，无物体则预测为背景**  

## 2 数据预处理  

SSD 标注数据如图所示：包含了原始图像及每个目标的坐标和类别  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/source_image.jpg?raw=true)
<center> **Figure 4** </center >
本文对训练数据做了 data augmentation，用来增加训练数据，提高MAP。  
用 SSD300 举例，每一张训练图像，随机的进行如下几种选择：  

1 原始图像resize之后进行训练  

原始图像resize成 300×300大小的图像，同样图像中目标标注的坐标也需要进行同样的转换  

2 zoom out 之后的图像进行训练  

zoom out 将一张图缩小, 并进行padding后作为输入, 有利于预测小物体，如下图所示  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/resize_canvus_image.jpg?raw=true)
<center> **Figure 5** </center >
这个过程一般先生成一张原图n倍大小的空图像，用image net 图像的均值[104, 117, 123]进行填充，然后将原始标注图像随机放到空图像的任意位置上，同时也变换目标标注坐标的位置，然后在按照 步骤 1 中 resize 的方法，将zoom out 之后的图像缩小成300×300大小。

3 翻转之后的图像进行训练  

如图所示：  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/mirror_source_image.jpg?raw=true)
<center> **Figure 6** </center >
4 随机的 crop 一个 patch 进行训练  

由于SSD是在不同特征层会预测不同大小的物体, 为了保证每个特征层都得到充分训练(特别是最后几层), 进行了图像的 crop , 这样相当于放大了一个物体  

对图像采样一个patch, 采样的条件是标注物体区域与采样后标注物体的区域要大于0.1 0.3 0.5 0.7 0.9，且保证物体中心点在crop区域  
crop后, ground truth将会被同样裁剪到crop图与ground truth重叠地区域，如下图所示，与上图相比，目标被放大：  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/corp_image_2.jpg?raw=true)
<center> **Figure 7** </center >

## 3 训练数据生成  

1 default box 生成  

一般来说，一个 CNN 网络中不同的 layers 有着不同尺寸的 感受野（receptive fields）。这里的感受野，指的是输出的 feature map 上的一个节点，其对应输入图像上尺寸的大小。大部分 CNN 网络在越深的层，feature map 的尺寸（size）会越来越小，对应的感受野也越大。  

因此，本文同时使用 lower feature maps、upper feature maps 来 predict detections。同时，feature map 中特定的位置，来负责图像中特定的区域，以及物体特定的尺寸。  

如图 Figure 1 所示，该文章分别在大小为[(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]的 6 个feature map 上生成 7308 个 default box，然后在这些生成的 default box 进行目标检测。每个feature map 上 default box 的尺寸可以用已下的公式计算：  

$ S_k = S_{min} + \frac{S_{max} - S_{min}}{m - 1}(k - 1) , k \in [1,m] $  

其中，$ S_{min} = 0.2 $,$ S_{max} = 0.95 $，意味着最底层的尺度为 0.2\*300,最高层的尺度为 0.95\*300,再用不同的aspect ratio 来生成 default boxes，用 $a_r$ 来表示：$ a_r = {1,2,3,1/2,1/3} $,则每个default box 的尺度就可以计算出来：  

$ w_k^a = s_k \sqrt {a_r} $, $ h_k^a = s_k / \sqrt {a_r} $  

对于aspect ratio 为 1 时，还增加了一个 default box，这个 box 的 scale 是 $ S_k = \sqrt {S_k S_{k+1}} $。最终在每个 feature map location 上，有 6 个 default boxes。  

如图 Figure 2 所示，在每个位置上就可以生成不同尺寸的default boxes。这些 default boxes 在定义好 feature map 的大小和 aspect ratio 之后，就固定不变，只需要生成一次。

2 default box 标注生成  

监督学习的训练关键是人工标注的label，对于包含default box的网络模型关键点就是如何把标注信息(ground true box,ground true category)映射到（default box上）。给定输入图像以及每个物体的 ground truth,首先找到每个ground true box对应的default box中IOU最大的作为（与该ground true box相关的匹配）正样本。然后，在剩下的default box中找到那些与任意一个ground truth box 的 IOU 大于0.5的default box作为（与该ground true box相关的匹配）正样本。 一个 ground truth 可能对应多个正样本default box 而不再像MultiBox那样只取一个IOU最大的default box。其他的作为负样本（每个default box要么是正样本box要么是负样本box）。  

总结起来就是:  
default box对应的是输入图像上固定的矩形区域  
一个 default box 对应一个输出  
default box 位置与输出节点的位置相对应, default box 大小与输出节点位于网络哪一层有关, default box 有固定的长宽比  
如果 default box 与物体 iou 大于0.5, 那么输出将会输出该物体类别，同时这个default box 对应的坐标就是这个物体的坐标标注  

3 Hard negative mining  

在生成一系列的 default boxes 之后，会产生很多个符合 ground truth box 的 boxes，但同时，不符合 ground truth boxes 也很多，而且这个 negative boxes，远多于 positive boxes。这会造成 negative boxes、positive boxes 之间的不均衡。训练时难以收敛。  
因此，本文采取，先将每一个物体位置上对应 default boxes 是 negative 的 boxes 进行排序（根据confidence loss对所有的box进行排序）。选择最高的几个，保证最后 negatives、positives 的比例在 3:1。  
本文通过实验发现，这样的比例可以更快的优化，训练也更稳定。


## 4 损失函数  

SSD损失函数分为两个部分：对应默认框的位置loss（loc）和类别置信度loss（conf）。定义 $x_{ij}^{p}= \{ 1,0 \} $ 为第 i 个默认框和对应的第 j个 ground truth box ，相应的类别为 p。目标函数定义为：  

$L(x,c,l,g)=\frac{1}{N}(L_{conf}(x,c)+\alpha L_{loc}(x,l,g))$  

其中，N为匹配的默认框。如果N=0，loss为零。$L_{conf}$ 为预测框 l 和 ground truth box g 的 Smooth L1 loss，类似 faster rcnn 中的方法， 该文章回归default bounding box (d) 中心点 (cx, cy) 和 width (w) and height (h) 的offsets。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/SSD5.jpg?raw=true)

## 参考  

- 1 [SSD: Single Shot MultiBox Detector](https://arxiv.org/abs/1512.02325)  
- 2 [论文阅读：SSD: Single Shot MultiBox Detector](http://lib.csdn.net/article/deeplearning/45325)  
- 3 [检测任务专题1: SSD在训练什么](https://zhuanlan.zhihu.com/p/29410169)  
- 4 [SSD caffe 源码](https://github.com/weiliu89/caffe/tree/ssd)  
- 5 [SSD tensorflow 源码](https://github.com/balancap/SSD-Tensorflow)  